SQL - Simple Queries
======


Python can simulate working with a SQL database.  This is done using SQLite.

In [1]:
MY_NAME = "Chris Phillips" # <-- Your name here

In [2]:
import sqlite3
conn = sqlite3.connect('simple_demo.db') # Name of the database to connect to.  
                                         # This will open or create a similiarly named file.

c = conn.cursor()

In [3]:
# Drop existing table (if present)
c.execute("DROP TABLE IF EXISTS users")

# Create users table
#c.execute("CREATE TABLE users(USER_ID INTEGER NOT NULL, NAME TEXT, NUM_FRIENDS INTEGER)")
c.execute("CREATE TABLE users(USER_ID INTEGER NOT NULL, NAME VARCHAR(20), NUM_FRIENDS INTEGER, INTEREST VARCHAR(20))")

# Populate
c.execute("INSERT INTO users VALUES (0, 'Hero', 0, 'SQL')")
c.execute("INSERT INTO users VALUES (1, 'Dunn', 2, 'SQL')")
c.execute("INSERT INTO users VALUES (2, 'Sue', 3, 'NoSQL')")

# Additional users to add
additional_users = [(3, "Chi", 3, 'SQL'), (4, "Thor", 3,'SQL'), (5, "Clive", 2,'SQL'), (6, "Hicks", 3,'SQL'), (7, "Devin", 2,'SQL'),
                    (8, "Kate", 2,'SQL'), (9, "Klein", 3,'SQL'), (10, "Jen", 1,'SQL')]

c.executemany("INSERT INTO users VALUES (?,?,?,?)", additional_users)

Display the total number of friends, the distinct number of interests, and all of the user's names.

In [4]:
### Solution ###
c.execute("SELECT SUM(num_friends) FROM users")
print c.fetchall()

c.execute("SELECT name FROM users")
print c.fetchall()

c.execute("SELECT COUNT(*) FROM (SELECT DISTINCT interest FROM users)")
print c.fetchall()

[(24,)]
[(u'Hero',), (u'Dunn',), (u'Sue',), (u'Chi',), (u'Thor',), (u'Clive',), (u'Hicks',), (u'Devin',), (u'Kate',), (u'Klein',), (u'Jen',)]
[(2,)]


What if we wanted to capture more than one interest per user.  One table will not cut it.  Let's add a user interests table and we can map many interest onto a user by user id.  

In [5]:
# Drop existing table (if present)
c.execute("DROP TABLE IF EXISTS users")

# Create users table
#c.execute("CREATE TABLE users(USER_ID INTEGER NOT NULL, NAME TEXT, NUM_FRIENDS INTEGER)")
c.execute("CREATE TABLE users(USER_ID INTEGER NOT NULL, NAME VARCHAR(20), NUM_FRIENDS INTEGER)")

# Populate
c.execute("INSERT INTO users VALUES (0, 'Hero', 0)")
c.execute("INSERT INTO users VALUES (1, 'Dunn', 2)")
c.execute("INSERT INTO users VALUES (2, 'Sue', 3)")

# Additional users to add
additional_users = [(3, "Chi", 3), (4, "Thor", 3), (5, "Clive", 2), (6, "Hicks", 3), (7, "Devin", 2),
                    (8, "Kate", 2), (9, "Klein", 3), (10, "Jen", 1)]

c.executemany("INSERT INTO users VALUES (?,?,?)", additional_users)

c.execute("DROP TABLE IF EXISTS user_interests")

# Create user interests table
c.execute("CREATE TABLE user_interests(USER_ID INTEGER NOT NULL, INTEREST TEXT)")

# Populate
c.executemany("INSERT INTO user_interests VALUES(?,?)", [(0, "SQLa"), (1, "NoSQL"), (2, "SQLa"), (3, "NoSQL")])
c.executemany("INSERT INTO user_interests VALUES(?,?)", [(0, "Basket Weaving"), (0, "Data Science Stuff")])


Display the number of interests by user_id.  Display each user's name along side one of thier respective interests.

In [6]:
### Solution
c.execute("SELECT user_id, COUNT(interest) FROM user_interests GROUP BY user_id")
print c.fetchall()

c.execute("SELECT name, interest FROM users, user_interests WHERE users.user_id = user_interests.user_id")
print c.fetchall()

[(0, 3), (1, 1), (2, 1), (3, 1)]
[(u'Hero', u'Basket Weaving'), (u'Hero', u'Data Science Stuff'), (u'Hero', u'SQLa'), (u'Dunn', u'NoSQL'), (u'Sue', u'SQLa'), (u'Chi', u'NoSQL')]


You might notice that there is a potential headache here.  If it was later realized that 'SQLa' should be 'SQL', there could many updates to make.  First, let's go ahead and dump the interests.  Then do an update.

In [7]:
### Soluction

c.execute("UPDATE user_interests SET interest = 'SQL' WHERE interest = 'SQLa'")
c.execute("SELECT * FROM user_interests")


Let's create an additional table.  We will then have a table for users, a table for interests and a table that maps between the two.  

In [8]:
# Drop existing table (if present)
c.execute("DROP TABLE IF EXISTS user")

# Create users table
#c.execute("CREATE TABLE users(USER_ID INTEGER NOT NULL, NAME TEXT, NUM_FRIENDS INTEGER)")
c.execute("CREATE TABLE user(USER_ID INTEGER NOT NULL, NAME VARCHAR(20), NUM_FRIENDS INTEGER)")

# Populate
c.execute("INSERT INTO user VALUES (0, 'Hero', 0)")
c.execute("INSERT INTO user VALUES (1, 'Dunn', 2)")
c.execute("INSERT INTO user VALUES (2, 'Sue', 3)")
c.execute("INSERT INTO user VALUES (3, 'Chi', 3)")

c.execute("DROP TABLE IF EXISTS interest")
# Create interests table
c.execute("CREATE TABLE interest(INTEREST_ID INTEGER NOT NULL, INTEREST TEXT)")

# Populate
c.executemany("INSERT INTO interest VALUES(?,?)", [(0, "SQL"), (1, "NoSQL")])
c.executemany("INSERT INTO interest VALUES(?,?)", [(2, "Basket Weaving"), (3, "Data Science Stuff")])

c.execute("DROP TABLE IF EXISTS has_interest_in")
# Create has_interest_in table
c.execute("CREATE TABLE has_interest_in (USER_ID INTEGER NOT NULL, INTEREST_ID INTEGER NOT NULL)")


Populate the has_interest_in table so that it contains the same mapping of users to interests as before.

In [9]:
# [(Hero, "SQL"), (Dunn, "NoSQL"), (Sue, "SQL"), (Chi, "NoSQL")])
# [(Hero, "Basket Weaving"), (Hero, "Data Science Stuff")])

### Solution

c.executemany("INSERT INTO has_interest_in VALUES(?, ?)", [(0,0), (1,1), (2,0), (3,1), (0,2), (0,3)])

In [10]:
c.execute("SELECT * FROM has_interest_in")
print c.fetchall()

[(0, 0), (1, 1), (2, 0), (3, 1), (0, 2), (0, 3)]


Repeat some of the previous queries with this new schema.  In particular, display the number of interests by user_id, display each user's name along side one of thier respective interests, total number of distinct interests.

Also display the top 3 interests by count in descending order. 

In [14]:
### Solution

# number of interests by user_id
c.execute("SELECT user_id, COUNT(interest_id) FROM has_interest_in GROUP BY user_id")
print c.fetchall()

# user name along side each respective interest
c.execute("SELECT name, interest FROM user, interest, has_interest_in WHERE user.user_id = has_interest_in.user_id AND interest.interest_id = has_interest_in.interest_id")
print c.fetchall()

# total number of distinct interest
c.execute("SELECT COUNT(*) FROM (SELECT DISTINCT interest_id FROM interest)")
print c.fetchall()

# top 3 interests by count in descending order
c.execute("SELECT interest, COUNT(has_interest_in.interest_id) AS cnt FROM has_interest_in, interest WHERE interest.interest_id = has_interest_in.interest_id GROUP BY has_interest_in.interest_id ORDER BY cnt DESC LIMIT 3")
print c.fetchall()


[(0, 3), (1, 1), (2, 1), (3, 1)]
[(u'Hero', u'SQL'), (u'Hero', u'Basket Weaving'), (u'Hero', u'Data Science Stuff'), (u'Dunn', u'NoSQL'), (u'Sue', u'SQL'), (u'Chi', u'NoSQL')]
[(4,)]
[(u'SQL', 2), (u'NoSQL', 2), (u'Basket Weaving', 1)]


In [12]:
# number of interests by user name
c.execute("SELECT name, COUNT(interest_id) FROM has_interest_in,user WHERE user.user_id = has_interest_in.user_id GROUP BY has_interest_in.user_id")
print c.fetchall()

[(u'Hero', 3), (u'Dunn', 1), (u'Sue', 1), (u'Chi', 1)]
